In [ ]:
import pandas
import operator
import numpy
import matplotlib.pyplot as pyplot
import datetime
import plotly.express as px
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash

In [ ]:
yesterday = (datetime.date.today() - datetime.timedelta(days = 1)).strftime("%m-%d-%Y")
print("Yesterday: ", yesterday)
data = pandas.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + yesterday + '.csv')

In [ ]:
print("Before cleaning rows: ", data.shape[0])
data["Lat"].replace("", numpy.nan, inplace=True)
data.dropna(subset=["Lat"], inplace=True)
print("After cleaning rows: ", data.shape[0])

In [ ]:
unique_countries =  list(data['Country_Region'].unique())

country_confirmed_cases = []
country_death_cases = [] 
country_active_cases = []
country_recovery_cases = []
country_incidence_rate = [] 
country_mortality_rate = [] 

no_cases = []
i = 0
while i < len(unique_countries):
    cases = data[data['Country_Region']==unique_countries[i]]['Confirmed'].sum()
    if cases > 0:
        country_confirmed_cases.append(cases)
    else:
        no_cases.append(unique_countries[i])
    i += 1

i = 0
while i < len(no_cases):
    unique_countries.remove(no_cases[i])
    i += 1
    
unique_countries = [k for k, v in sorted(zip(unique_countries, country_confirmed_cases), key=operator.itemgetter(1), reverse=True)]
i = 0
while i < len(unique_countries):
    country_confirmed_cases[i] = data[data['Country_Region']==unique_countries[i]]['Confirmed'].sum()
    country_death_cases.append(data[data['Country_Region']==unique_countries[i]]['Deaths'].sum())
    country_recovery_cases.append(data[data['Country_Region']==unique_countries[i]]['Recovered'].sum())
    country_active_cases.append(data[data['Country_Region']==unique_countries[i]]['Active'].sum())
    country_incidence_rate.append(data[data['Country_Region']==unique_countries[i]]['Incident_Rate'].sum())
    country_mortality_rate.append(country_death_cases[i]/country_confirmed_cases[i])
    i += 1

In [ ]:
country_df = pandas.DataFrame({
    'Country': unique_countries, 
    'No. of Confirmed Cases': country_confirmed_cases,
    'No. of Deaths': country_death_cases, 
    'No. of Recoveries' : country_recovery_cases, 
    'No. of Active Cases' : country_active_cases, 
    'Incidence Rate' : country_incidence_rate,
    'Mortality Rate': country_mortality_rate
})

# country_df.style.background_gradient(cmap='Greens')

In [ ]:
def plot_bar_graphs(x, y, title):
    pyplot.figure(figsize=(16, 12))
    pyplot.barh(x, y)
    pyplot.title(title, size=20)
    pyplot.xticks(size=20)
    pyplot.yticks(size=20)
    pyplot.show()

def plot_pie_charts(x, y, title):
    c = ['lightcoral', 'rosybrown', 'sandybrown', 'navajowhite', 'gold',
        'khaki', 'lightskyblue', 'turquoise', 'lightslategrey', 'thistle', 'pink']
    pyplot.figure(figsize=(20,15))
    pyplot.title(title, size=20)
    pyplot.pie(y, colors=c,shadow=True, labels=y)
    pyplot.legend(x, loc='best', fontsize=12)
    pyplot.show()

In [ ]:
visual_unique_countries = [] 
visual_confirmed_cases = []
others = numpy.sum(country_confirmed_cases[10:])

i = 0
while i < len(country_confirmed_cases[:10]):
    visual_unique_countries.append(unique_countries[i])
    visual_confirmed_cases.append(country_confirmed_cases[i])
    i += 1
    
visual_unique_countries.append('Others')
visual_confirmed_cases.append(others)

In [ ]:
# plot_bar_graphs(visual_unique_countries, visual_confirmed_cases, '# of Covid-19 Confirmed Cases in Countries/Regions')
# plot_pie_charts(visual_unique_countries, visual_confirmed_cases, 'Covid-19 Confirmed Cases per Country')

In [ ]:
fig_df = pandas.DataFrame({
    'Country': visual_unique_countries,
    'Confirmed Cases': visual_confirmed_cases
})
fig_bar = px.bar(
    fig_df,
    x = fig_df['Confirmed Cases'],
    y = fig_df['Country'],
    labels = {
        "x": "Confirmed Cases",
        "y": "Country"
    },
    title = "# of Covid-19 Confirmed Cases in Countries/Regions",
    orientation = "h"
)
fig_pie = px.pie(
    fig_df,
    values = "Confirmed Cases",
    names = "Country",
    title = "Covid-19 Confirmed Cases per Country"
)

In [ ]:
app = JupyterDash(__name__)
app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),
    html.Div(children='''
        Dash: A web application framework for your data.
    '''),
    html.Div(
        children = dcc.Graph(
            id = "bar_no_confirmed",
            figure = fig_bar
        )
    ),
    html.Div(
        children = dcc.Graph(
            id = "pie_no_confirmed",
            figure = fig_pie
        )
    )
])

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)